In [1]:
import numpy as np
from math import exp

In [2]:

class NeuralNetwork:
    def __init__(self, inputSize,outputSize,outputLayerActivation='sigmoid'):

        self.weights = [np.random.randn(inputSize, outputSize) * np.sqrt(2. / inputSize)]
        self.bias = [np.zeros((1, outputSize))]
        self.layers = 0
        self.activations = [outputLayerActivation]

    def addLayer(self,size,activation='relu'):
        self.layers +=1
        self.weights.append(np.random.randn(size, self.weights[-1].shape[1]) * np.sqrt(2. / size))
        self.weights[-2] = np.random.randn(self.weights[-2].shape[0], self.weights[-1].shape[0]) * np.sqrt(2. / self.weights[-1].shape[0])

        self.bias.append(np.zeros((1,self.weights[-1].shape[1])))
        self.bias[-2] = np.zeros((1,size))        
        
        self.activations.insert(-2,activation)

    

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return np.where(x <= 0, 0, 1)
    
    def computeLoss(self, y_true, y_pred):
        epsilon = 1e-10
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
        m = y_true.shape[0]
        loss = -1 / m * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

        return loss



    def predict(self, X):
        self.output = [X]
       
        for i in range(self.layers+1):
            self.z = np.dot(self.output[-1],self.weights[i]) + self.bias[i]
            self.output.append(eval(f"self.{self.activations[i]}(self.z)"))
            
        return self.output[-1]
    
    def backward(self, X, y, learningRate=0.01):
        m = X.shape[0]

        dz = [self.output[-1] - y]
        dw = [(1 / m) * np.dot(self.output[-2].T, dz[0])]
        db = [(1 / m) * np.sum(dz[0], axis=0, keepdims=True)]

        for i in range(self.layers,0,-1):
            dz = [np.dot(dz[0], self.weights[i].T) * eval(f"self.{self.activations[i-1]}_derivative(self.output[i])")] + dz
            dw = [(1 / m) * np.dot(self.output[i-1].T, dz[0])] + dw
            db = [(1 / m) * np.sum(dz[0], axis=0, keepdims=True)] + db
            

        for i in range(self.layers+1):
            
            self.weights[i] -= learningRate * dw[i]
            self.bias[i] -= learningRate * db[i]
            


    def fit(self,X,y,epochs,learningRate=0.01,verbose=True,verboseInterval=100):
        loss = []
    
        for epoch in range(epochs):
           
            if epoch%verboseInterval == 0 and verbose:
                print(f"\nEpoch: {epoch}\n|",end='')
                print("====================",end='')

            pred = self.predict(X)
            loss.append(self.computeLoss(y,pred))
            self.backward(X, y,learningRate)

            if epoch%verboseInterval == 0 and verbose:
                print(f"|   Loss: {loss[-1]}")
            

        if verbose:
            print('')
            
        return loss
          


In [3]:
inputSize = 3
hiddenSize1 = 4
hiddenSize2 = 4
outputSize = 2

options = np.array([[0, 0 ,0], [0, 0, 1]])
outputs = np.array([[1,0], [0,1]])

# Generate random training data
i = np.random.choice(2, size=1000)
X = options[i]
y = outputs[i]

In [4]:
from NeuroFlow import Sequential

model = Sequential(inputSize,outputSize,outputLayerActivation='softmax')





In [5]:
loss = model.fit(X,y,100000,verboseInterval=50000)


model.save("F.file")



Epoch: 0
|====================|   Loss: 2.2687760019353984

Epoch: 50000
|====================|   Loss: 0.010250072710037188

Training Complete


In [9]:
from NeuroFlow import load_model


mod = loadModel("F.file")
new_data = np.array([[0, 1 ,0], [0, 0, 1],[0,1,1]])
predictions = mod.predict(new_data)
print("Predictions:", predictions)

Predictions: [[9.93161293e-01 6.83870699e-03]
 [2.01344979e-03 9.97986550e-01]
 [8.97710406e-04 9.99102290e-01]]
